# Show Status of Download

In [1]:
import os.path
import pandas as pd

from bokeh.core.properties import value
from bokeh.io import show, output_file, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, FactorRange, HoverTool
output_notebook()

Loading BokehJS ...

Group by either month/year or date.

In [2]:
## User Params
# grby = ['year','month']
grby = ['date']

Get the details about items to download from `img_id_file` and the loaded items from the destination folder `img_folder`.

In [3]:
img_folder = './data'
# img_id_file = 'pathum_2019-01-20_2019-01-21.txt'
# img_id_file = 'pathum_2019-02-22_2019-02-23.txt'
# img_id_file = 'pathum_2019-03-23_2019-03-24.txt'
# img_id_file = 'pathum_2019-04-19_2019-04-20.txt'
# img_id_file = 'pathum_2019-05-17_2019-05-18.txt'
# img_id_file = 'pathum_2019-06-23_2019-06-24.txt'
# img_id_file = 'pathum_2019-07-20_2019-07-21.txt'
# img_id_file = 'pathum_2019-08-19_2019-08-20.txt'
img_id_file = 'pathum_2019-09-26_2019-09-27.txt'

img_id_file = 'id_files/' + img_id_file

In [4]:
from planet_utils import get_dest_df, read_img_id_file
search_id, item_type, asset_type, img_ids = read_img_id_file(img_id_file)
df = get_dest_df(img_ids, img_folder, item_type, asset_type)
df.head()

,id,year,month,date,dest
0,2706394_4745521_2019-09-26_103d,2019,09,2019-09-26,./data/PSOrthoTile/visual/2019/09/2019-09-26/2...
1,2706394_4745520_2019-09-26_103d,2019,09,2019-09-26,./data/PSOrthoTile/visual/2019/09/2019-09-26/2...
2,2706394_4745621_2019-09-26_103d,2019,09,2019-09-26,./data/PSOrthoTile/visual/2019/09/2019-09-26/2...
3,2706394_4745620_2019-09-26_103d,2019,09,2019-09-26,./data/PSOrthoTile/visual/2019/09/2019-09-26/2...
4,2705267_4745523_2019-09-26_1004,2019,09,2019-09-26,./data/PSOrthoTile/visual/2019/09/2019-09-26/2...


Get all stats we need: total no. of images, no. of loaded images, and no. of remaining images.
(Run this cell to get the updated result)

In [5]:
# get all stats: no. of total, loaded, not loaded
df['exist'] = df.dest.apply(lambda x: int(os.path.exists(x)))

total = df.groupby(grby)[["id"]].count().values.transpose().tolist()[0]
loaded = df.groupby(grby)[["exist"]].sum().values.transpose().tolist()[0]
notload = [t-l for t, l in zip(total, loaded)]

Show progress using Bokeh

In [6]:
dates = list(df.groupby(grby).groups.keys())
status = ["Loaded", "Not Load"]
colors = ["#718dbf", "#e84d60"]

data = {'date'    : dates,
        'Loaded'  : loaded,
        'Not Load': notload}

p = figure(y_range=FactorRange(*dates), title="Download Status")

p.hbar_stack(status, y='date', height=0.9, color=colors, source=data, legend=status)

p.x_range.start = 0
p.y_range.range_padding = 0.1
p.ygrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_right"
p.legend.orientation = "horizontal"

tooltips = HoverTool(
    tooltips=[
        ("Loaded", "@Loaded"),
        ("Not Load", "@{Not Load}"),
        ("Date", "@date")
        ]
    )
p.add_tools(tooltips)

show(p)